# 🧠 HIM - Hyper-Introspective Model (7M Parameters)

**Un modèle Transformer ultra-compact avec mécanismes cognitifs avancés**

- 7.16M paramètres (99.66% de réduction vs 2.1B)
- 4 mécanismes cognitifs: Mémoire, Attention GQA, Multi-vitesse, Raisonnement
- Entraînement sur WikiText-103
- Architecture inspirée de Llama-2, PaLM, Phi-1

---

## 📋 Instructions

1. **Activer GPU**: Runtime → Change runtime type → GPU (T4)
2. **Exécuter toutes les cellules** dans l'ordre
3. **Entraînement**: ~6-8 heures pour 30 epochs sur GPU T4
4. **Télécharger le modèle** à la fin

---

## 1️⃣ Configuration et Installation

In [1]:
# Vérifier le GPU
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Installer les dépendances
!pip install -q torch transformers datasets tokenizers tqdm evaluate sacrebleu bert-score nltk matplotlib seaborn


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Cloner le repository (ou uploader vos fichiers)
import os
from google.colab import drive

# Option 1: Cloner depuis GitHub (si vous avez pushé le code)
# !git clone https://github.com/votre-username/HIM.git
# %cd HIM

# Option 2: Monter Google Drive et copier les fichiers
drive.mount('/content/drive')
print("\n⚠️ IMPORTANT: Uploadez votre dossier HIM dans Google Drive")
print("Ensuite, décommentez et modifiez la ligne suivante:")
# !cp -r /content/drive/MyDrive/HIM /content/
# %cd /content/HIM

In [ ]:
# Option 3: Upload manuel des fichiers (si pas de GitHub/Drive)
# Créer la structure de dossiers
!mkdir -p src model_artifacts/tokenizer

print("📁 Uploadez maintenant les fichiers suivants:")
print("  src/components.py")
print("  src/model.py")
print("  src/dataset.py")
print("  src/train.py")
print("  src/optimizer.py")
print("  train_tokenizer.py")
print("  main.py")
print("\nUtilisez le panneau de gauche (Files) pour uploader")

## 2️⃣ Entraînement du Tokenizer

In [ ]:
# Entraîner le tokenizer BPE (30K tokens)
!python train_tokenizer.py --vocab_size 30000 --dataset_name wikitext --config_name wikitext-103-v1

In [ ]:
# Vérifier le tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('model_artifacts/tokenizer')
print(f"✅ Tokenizer chargé: {len(tokenizer)} tokens")

# Test
test_text = "The quick brown fox jumps over the lazy dog. 🚀"
tokens = tokenizer.encode(test_text)
decoded = tokenizer.decode(tokens)
print(f"\nTest: '{test_text}'")
print(f"Tokens: {tokens}")
print(f"Décodé: '{decoded}'")

## 3️⃣ Entraînement du Modèle

### Configuration recommandée pour GPU T4 (gratuit)

In [ ]:
# Configuration d'entraînement
CONFIG = {
    # Architecture (7.16M params)
    'embed_dim': 160,
    'num_heads': 8,
    'num_kv_heads': 2,  # GQA
    'num_encoder_layers': 10,
    'dim_feedforward': 320,
    'mem_slots': 16,
    'mem_rank': 16,
    'reasoning_tokens': 64,
    
    # Entraînement
    'num_epochs': 30,
    'batch_size': 16,  # Plus élevé sur GPU
    'accumulation_steps': 4,  # Batch effectif = 64
    'learning_rate': 1e-3,
    'grad_clip_value': 1.0,
    'dropout': 0.1,
    'label_smoothing': 0.1,
    
    # Optimisations
    'fp16': True,  # Mixed precision
    'use_cognitive_losses': True,
    'smart_training': False,  # Désactivé pour Colab
    'save_steps': 5000,
    
    # Post-training
    'pruning_amount': 0.2,
    'fine_tune_epochs': 2
}

print("📊 Configuration d'entraînement:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

In [ ]:
# Lancer l'entraînement
!python main.py \
    --task_type wikitext \
    --embed_dim {CONFIG['embed_dim']} \
    --num_heads {CONFIG['num_heads']} \
    --num_kv_heads {CONFIG['num_kv_heads']} \
    --num_encoder_layers {CONFIG['num_encoder_layers']} \
    --dim_feedforward {CONFIG['dim_feedforward']} \
    --mem_slots {CONFIG['mem_slots']} \
    --mem_rank {CONFIG['mem_rank']} \
    --reasoning_tokens {CONFIG['reasoning_tokens']} \
    --num_epochs {CONFIG['num_epochs']} \
    --batch_size {CONFIG['batch_size']} \
    --accumulation_steps {CONFIG['accumulation_steps']} \
    --learning_rate {CONFIG['learning_rate']} \
    --grad_clip_value {CONFIG['grad_clip_value']} \
    --dropout {CONFIG['dropout']} \
    --label_smoothing {CONFIG['label_smoothing']} \
    --fp16 \
    --use_cognitive_losses \
    --pruning_amount {CONFIG['pruning_amount']} \
    --fine_tune_epochs {CONFIG['fine_tune_epochs']} \
    --save_steps {CONFIG['save_steps']}

## 4️⃣ Monitoring et Visualisation

In [ ]:
# Visualiser les métriques d'entraînement
import json
import matplotlib.pyplot as plt

# Charger les métriques
with open('model_artifacts/training_metrics.json', 'r') as f:
    metrics = json.load(f)

# Plot perplexité
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Perplexité
axes[0, 0].plot(metrics['train_perplexity'], label='Train')
axes[0, 0].plot(metrics['val_perplexity'], label='Validation')
axes[0, 0].set_title('Perplexité')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Perplexité')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Loss
axes[0, 1].plot(metrics['train_loss'], label='Train')
axes[0, 1].plot(metrics['val_loss'], label='Validation')
axes[0, 1].set_title('Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# BERTScore
axes[1, 0].plot(metrics['val_bertscore'], label='BERTScore F1')
axes[1, 0].set_title('BERTScore (Qualité Sémantique)')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Score')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Distinct-2
axes[1, 1].plot(metrics['val_distinct_2'], label='Distinct-2')
axes[1, 1].set_title('Distinct-2 (Diversité)')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Score')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Courbes d'entraînement sauvegardées dans 'training_curves.png'")

In [ ]:
# Visualiser les métriques cognitives
import matplotlib.pyplot as plt
import json

with open('model_artifacts/cognitive_metrics.json', 'r') as f:
    cog_metrics = json.load(f)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Memory utilization
axes[0].plot(cog_metrics['memory_utilization'])
axes[0].set_title('Utilisation Mémoire (%)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('% Slots Actifs')
axes[0].grid(True)

# Reasoning diversity
axes[1].plot(cog_metrics['reasoning_diversity'])
axes[1].set_title('Diversité Raisonnement')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Score')
axes[1].grid(True)

# Routing balance
axes[2].plot(cog_metrics['avg_slow_preference'], label='Slow Path')
axes[2].plot(cog_metrics['avg_fast_preference'], label='Fast Path')
axes[2].set_title('Balance Routage')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Préférence (%)')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.savefig('cognitive_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Métriques cognitives sauvegardées dans 'cognitive_metrics.png'")

## 5️⃣ Test d'Inférence

In [ ]:
# Tester le modèle entraîné
import torch
from transformers import AutoTokenizer
from src.model import NLDirectResponse

# Charger le modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('model_artifacts/tokenizer')

checkpoint = torch.load('model_artifacts/nl_direct_response_model_best.pth', map_location=device)
config = checkpoint['config']

model = NLDirectResponse(
    vocab_size=config['vocab_size'],
    embed_dim=config['embed_dim'],
    num_heads=config['num_heads'],
    num_kv_heads=config['num_kv_heads'],
    num_encoder_layers=config['num_encoder_layers'],
    dim_feedforward=config['dim_feedforward'],
    max_answer_len=config['max_answer_len'],
    dropout=0.0,
    mem_slots=config['mem_slots'],
    mem_rank=config['mem_rank'],
    reasoning_tokens=config['reasoning_tokens']
)

model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

print("✅ Modèle chargé avec succès!")
print(f"Epoch: {checkpoint.get('epoch', 'N/A')}")
print(f"Best Val Loss: {checkpoint.get('best_val_loss', 'N/A'):.4f}")

In [ ]:
# Générer du texte
def generate_text(prompt, temperature=0.8, top_k=50, top_p=0.9):
    model.reset_memory()
    
    inputs = tokenizer(prompt, return_tensors='pt', padding='max_length',
                      max_length=512, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        logits = model(input_ids, src_padding_mask=attention_mask)
    
    # Sampling avec température
    logits = logits[0] / temperature
    
    # Top-k filtering
    if top_k > 0:
        indices_to_remove = logits < torch.topk(logits, top_k, dim=-1)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
    
    # Échantillonnage
    probs = torch.softmax(logits, dim=-1)
    predicted_ids = torch.multinomial(probs, num_samples=1).squeeze(-1)
    
    generated_text = tokenizer.decode(predicted_ids, skip_special_tokens=True)
    return generated_text

# Tests
test_prompts = [
    "The history of artificial intelligence began in",
    "Machine learning is a field of study that",
    "In the future, technology will",
    "The most important discovery in science was"
]

print("🎨 Génération de texte:\n")
for prompt in test_prompts:
    generated = generate_text(prompt)
    print(f"Prompt: {prompt}")
    print(f"Généré: {generated}")
    print("-" * 80)

## 6️⃣ Téléchargement des Résultats

In [ ]:
# Compresser tous les artefacts
!zip -r HIM_trained_model.zip model_artifacts/ *.png

print("✅ Fichiers compressés dans HIM_trained_model.zip")
print("\nContenu:")
!unzip -l HIM_trained_model.zip

In [ ]:
# Télécharger le fichier
from google.colab import files

files.download('HIM_trained_model.zip')
print("📥 Téléchargement démarré!")

In [ ]:
# Ou sauvegarder dans Google Drive
!cp HIM_trained_model.zip /content/drive/MyDrive/
print("✅ Modèle sauvegardé dans Google Drive")

## 📊 Résumé Final

In [ ]:
# Afficher le résumé final
import json

print("=" * 80)
print("HIM - RÉSUMÉ DE L'ENTRAÎNEMENT")
print("=" * 80)

# Charger les métriques finales
with open('model_artifacts/training_metrics.json', 'r') as f:
    metrics = json.load(f)

print(f"\n📈 Métriques Finales:")
print(f"  Epochs entraînés: {len(metrics['train_loss'])}")
print(f"  Best Val Loss: {min(metrics['val_loss']):.4f}")
print(f"  Best Perplexity: {min(metrics['val_perplexity']):.2f}")
print(f"  Best BERTScore: {max(metrics['val_bertscore']):.4f}")
print(f"  Best Distinct-2: {max(metrics['val_distinct_2']):.4f}")

with open('model_artifacts/cognitive_metrics.json', 'r') as f:
    cog_metrics = json.load(f)

print(f"\n🧠 Mécanismes Cognitifs:")
print(f"  Utilisation mémoire finale: {cog_metrics['memory_utilization'][-1]:.1f}%")
print(f"  Diversité raisonnement: {cog_metrics['reasoning_diversity'][-1]:.4f}")
print(f"  Balance routage: Slow={cog_metrics['avg_slow_preference'][-1]:.1f}%, Fast={cog_metrics['avg_fast_preference'][-1]:.1f}%")

# Paramètres du modèle
total_params = sum(p.numel() for p in model.parameters())
print(f"\n⚙️ Architecture:")
print(f"  Paramètres totaux: {total_params:,}")
print(f"  Taille du modèle: ~{total_params * 4 / 1024 / 1024:.1f} MB (float32)")
print(f"  Taille quantisée: ~{total_params / 1024 / 1024:.1f} MB (int8)")

print("\n" + "=" * 80)
print("✅ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS!")
print("=" * 80)
print("\nFichiers générés:")
print("  - model_artifacts/nl_direct_response_model_best.pth (modèle)")
print("  - model_artifacts/nl_direct_response_model_quantized.pth (quantisé)")
print("  - model_artifacts/training_metrics.json (métriques)")
print("  - training_curves.png (visualisations)")
print("  - cognitive_metrics.png (mécanismes cognitifs)")